# Сбор данных

In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import numpy as np
import random
from datetime import date
import datetime
import time
import pandas as pd
import os

# константы
RANDOM_STATE = 5
DATA_PATH = '../data'
COOCIE_NB = '_csrf_token=322d41e9006e0d84d35ea2ba160ffb0edcaf8bde6af076a0; suid=d5b39f92ec07e14b5ede65193afae8dd.b7a4c96080bd260714d8bcd31af9fb4a; autoruuid=g671fe5f02tuv4stgtm003ft6ufjgtuf.cf4d48bce1d92127b1dc5420a6c012d3; from=direct; autoru_sso_blocked=1; Session_id=3:1730143729.5.0.1729626845318:0EzFWw:1e52.1.2:1|1130000067438128.-1.0.3:1729626845|61:10026972.494636.G47KJoJnrGleXdU3AV1aAbBSOIE; sessar=1.1195.CiC4kiZ9xgGyA9EKYOSeDcPovH-CImZprpSTDFCpFXMUfQ.JxIZidV8vdomSfagoec4_A7vSfKJYsUeVx-hqv85aII; yandex_login=daboldyrev@edu.hse.ru; ys=udn.cDpkYWJvbGR5cmV2QGVkdS5oc2UucnU%3D#c_chck.1929241363; i=tlhDxtDyQeSf93G4Sd2jN8QeRk4YamYpZXEiLrhQS7CLpw0+5vhEzZB3fpv6+DoEc80kD0DF7VDIIgjCNuTIizOlOu8=; yandexuid=2160422051564510217; my=YwA=; L=Xy5fSWVnBHIJSEN2UVlaRQ5hWW9bXVoGIFYzOSssLABdEC9SECxcJxsSVytC.1729626845.15927.340609.84c416984beb9e0cf870a4d6f92acb61; mda2_beacon=1730143729098; sso_status=sso.passport.yandex.ru:synchronized; autoru_sso_redirect_blocked=1; _ym_uid=1684353637548378497; yaPassportTryAutologin=1; _ym_isad=2; autoru_sid=123729954%7C1730143739148.7776000.b4c5nf1yViUEOZ-QftLYSw.mUjU3EqWVT1cPoNPBDcE6l2ZGA5w6n6ifxDp-_Y12Pc; _yasc=+J+VIii3pr3WbEOhoe+sjg44LQZA886gpGdnqTZiAIEC+O3+kkAC+EqS5keXehajStIA; popups-popup-guess-the-car-shown-count=1; _ym_d=1730145169; count-visits=4; from_lifetime=1730145175566; layout-config={"screen_height":1080,"screen_width":1920,"win_width":334,"win_height":983}; cycada=Crddb5c+sUwEnCqwO6jVWnYzs+eP5+y9mEaD7l6eSdg='

COOCIE_PC = '_ym_uid=1730325262421443420; _ym_isad=2; gdpr=0; _ym_visorc=b; spravka=dD0xNzMwMzI1Mjc3O2k9MTc4LjIzNC4xOS4yMDI7RD1GNjQ4OUE1NTVBQTBCMjE4MzAxOUU4RUREOTA1NkIyM0UxQTRGMTI1RDc0NTE1NTU1MzBEOTA4OUNFQzRBNDcwNzBFQjExRUU4QTVFNzUxNUUyNzRFNzFGOEE2REZFRDhCNzhCRUUzQkIzNjAwNjg0NkRGMDAxNUM1NkM2QjY0RkY3Qjg0RURDMUQ7dT0xNzMwMzI1Mjc3NDcwODA4MTA2O2g9ZmRkODI1Y2YzZjg1ZWI1MGFkMGU2NDRhMjExY2IzYjc=; suid=8ce6731f9a649540b81418dd7f7a116e.47f9f10efbd002c2a5df27708dee6294; _csrf_token=336e9598aa89701ea1d4a92ef74b8383aeb945805101384c; autoruuid=g6722ab1d22n1or4fd1vhlrpil0a28cb.27fdd35057ca4aa98e84995de580e2db; from=direct; autoru_sso_blocked=1; Session_id=3:1730325277.5.1.1661287478014:g-_qsg:7a.1.2:1|46441660.519755.2.2:519755|1130000067438128.-1.0.2:63503060.3:1724790538|61:10027023.161971.xI-XPxtAO96MYM_W4izayeiUh8Q; sessar=1.1195.CiAfLgyg5QFO9kv_v2Iwwkp6R5lIv6cePPCbuQfC79m-WA.NhwRXi6eFd_e185pyLkh8Ueus8HpDaU0f9XAxY7snJk; yandex_login=daboldyrev@edu.hse.ru; ys=udn.cDpkYWJvbGR5cmV2QGVkdS5oc2UucnU%3D#c_chck.2288135939; i=nzlq8Txl051mhytlolic3YKu6ZRZu7M23AglGQC2H6SChVI3I4dx9PEr/Dfs4y+y5EDaXVNKRdMgzgZN1dIUuL2Jc14=; yandexuid=8456723751661209048; L=AF8Ae2BiUUNFeWdaY0BiWHMBWQxYDglSIykrGSoFKgIoHhFWFCNAIBAtWhYf.1724955754.15868.39776.ae22e28e4e90f779e216b76750191ceb; mda2_beacon=1730325277687; sso_status=sso.passport.yandex.ru:synchronized; autoru_sso_redirect_blocked=1; crookie=Z1aKuGe6JLy4FKGXhIj3nnrehm9/YQjGl0VCNKR0jUKr7d2h3ivxaIYPu13wQXm619av9REd2gfP7/GSF9hkstsjJgk=; cmtchd=MTczMDMyNTI4MTY4NQ==; yaPassportTryAutologin=1; autoru_sid=123729954%7C1730325282210.7776000.jxWwPZ77fRVUIlfmhfmAtw.DItU0HaaGmmf9623BImFHi08A6jgdeS1mDF0pbX9qGc; popups-popup-guess-the-car-shown-count=1; _ym_d=1730325649; _yasc=pjX03g9KqviPlHjHOW4LvHXFlqUFi18/SLXJxQ60LnoY+pyRIcEArbgnLiJumj0F7zzKmA==; count-visits=5; from_lifetime=1730325650905; layout-config={"screen_height":900,"screen_width":1440,"win_width":876,"win_height":803}; cycada=Kx3ttLi+7hFZXHKh+gVBbnYzs+eP5+y9mEaD7l6eSdg='

## Настройки для подключения к сайту

Используем сведения из Headers при обращении к https://auto.ru/cars/used/ (зависят от браузера, компьютера), в том числе вводим cookies.

In [3]:
# cookie = COOCIE_PC # с компьютера
cookie = COOCIE_NB # с ноутбука

headers = {
    'authority': 'auto.ru',
    'method': 'GET',
    'path': '/cars/used/',
    'scheme': 'https',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-encoding': 'gzip, deflate, br, zstd',
    'accept-language': 'ru,en;q=0.9,en-GB;q=0.8,en-US;q=0.7',
    'cache-control': 'max-age=0',
    'cookie': cookie,
    'priority': 'u=0, i',
    'sec-ch-ua': '"Chromium";v="130", "Microsoft Edge";v="130", "Not?A_Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform':'"Windows"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36 Edg/130.0.0.0'
}

## Функции для сбора данных

In [2]:
# Определим функцию, которая делает паузу случайной длительности
def sleep():
    random_seconds = np.linspace(3, 10, 100)
    time.sleep(random.choice(random_seconds))

In [3]:
# Определим функцию, которая загружает справочник комплектаций
def open_complectation(file_name):
    complectations = []
    with open(file_name) as data:
        for record in data:
            res = {}
            for paars in record[1:-2].split(", '"):
                key, value = paars.split("':")
                value = value.replace("'","").strip()
                if value == 'None':
                    value = None
                res[key.replace("'","").strip()] = value
            complectations.append(res)
    return complectations

In [4]:
# Определим функцию, которая загружает комплектацию модели
# по адресу url_compl
def get_complectation(url_compl):

    # Немного подождали
    sleep()
    
    # загрузили страницу с данными по автомобилю
    page_compl = requests.get(url_compl, headers=headers)
    
    # получили иерархическое представление содержимого    
    soup_compl = BeautifulSoup(page_compl.content, 'html.parser')

    # Нашли нужный блок
    card_info = soup_compl.find('div', {'class': 'ModificationInfo-GiDD1'})
    
    # создали словарь
    row = {
        'url_compl': url_compl,
        'state_mark': None,
        'class_auto': None,
        'door_count': None,
        'seat_count': None,
        'long': None,
        'width': None,
        'height': None,
        'clearence': None,
        'v_bag': None,
        'v_tank': None,
        'curb_weight': None,
        'gross_weight': None,
        'front_brakes': None,
        'rear_brakes': None,
        'max_speed': None,
        'acceleration': None,
        'fuel_cons': None,
        'fuel_brand': None,
        'engine_loc1': None,
        'engine_loc2': None,
        'turbocharg': None,
        'max_torq': None,
        'cyl_count': None
        }
    
    # извлекли сведения о комплектации
    for li in card_info.find_all('li'): 
        li_span = li.find('span').text.strip()
        match li_span:
            case 'Страна марки':
                try:
                    row['state_mark'] = li.find_all('span')[1].text.strip().replace('\xa0',' ')
                except:
                    pass
            case 'Класс автомобиля':
                try:
                    row['class_auto'] = li.find_all('span')[1].text.strip().replace('\xa0',' ')
                except:
                    pass
            case 'Количество дверей':
                try:
                    row['door_count'] = li.find_all('span')[1].text.strip().strip()
                except:
                    pass
            case 'Количество мест':
                try:
                    row['seat_count'] = li.find_all('span')[1].text.strip().strip()
                except:
                    pass
            case 'Длина':
                try:
                    row['long'] = li.find_all('span')[1].text.strip().split()[0]
                except:
                    pass
            case 'Ширина':
                try:
                    row['width'] = li.find_all('span')[1].text.strip().split()[0]
                except:
                    pass
            case 'Высота':
                try:
                    row['height'] = li.find_all('span')[1].text.strip().split()[0]
                except:
                    pass
            case 'Клиренс':
                try:
                    row['clearence'] = li.find_all('span')[1].text.strip().split()[0]
                except:
                    pass
            case 'Объем багажника':
                try:
                    row['v_bag'] = li.find_all('span')[1].text.strip().split()[0]
                except:
                    pass
            case 'Объём топливного':
                try:
                    row['v_tank'] = li.find_all('span')[1].text.strip().split()[0]
                except:
                    pass
            case 'Снаряженная масса':
                try:
                    row['curb_weight'] = li.find_all('span')[1].text.strip().split()[0]
                except:
                    pass
            case 'Полная масса':
                try:
                    row['gross_weight'] = li.find_all('span')[1].text.strip().split()[0]
                except:
                    pass
            case 'Передние тормоза':
                try:
                    row['front_brakes'] = li.find_all('span')[1].text.strip().split()[0]
                except:
                    pass
            case 'Задние тормоза':
                try:
                    row['rear_brakes'] = li.find_all('span')[1].text.strip().split()[0]
                except:
                    pass
            case 'Максимальная скорость':
                try:
                    row['max_speed'] = li.find_all('span')[1].text.strip().split()[0]
                except:
                    pass
            case 'Разгон':
                try:
                   row['acceleration'] = li.find_all('span')[1].text.strip().split()[0]
                except:
                    pass
            case 'Расход топлива':
                try:
                    row['fuel_cons'] = li.find_all('span')[1].text.strip().split('/')[1]
                except:
                    pass
            case 'Марка топлива':
                try:
                    row['fuel_brand'] = li.find_all('span')[1].text.strip()
                except:
                    pass
            case 'Расположение двигателя':
                try:
                    row['engine_loc1'] = li.find_all('span')[1].text.strip().split(', ')[0]
                except:
                    pass
                try:
                    row['engine_loc2'] = li.find_all('span')[1].text.strip().split(', ')[1]
                except:
                    pass
            case 'Тип наддува':
                try:
                    row['turbocharg'] = li.find_all('span')[1].text.strip()
                except:
                    pass
            case 'Максимальный крутящий':
                try:
                    row['max_torq'] = li.find_all('span')[1].text.strip().split()[0]
                except:
                    pass
            case 'Количество цилиндров':
                try:
                    row['cyl_count'] = li.find_all('span')[1].text.strip()
                except:
                    pass
            case _:
                pass
    # передали словарь
    return row

In [5]:
# Определим функцию, которая загружает данные из объявления
# по адресу url_car
def get_car(url_car):    

    # Немного подождали
    sleep()
    
    # зададим год, в котором опубликованы объявления
    year = 2024
    
    # Зададим количество л.с. в 1 кВт
    HP_per_kW = 1.3596216173
    
    # подготовили словарь для месяцев
    dict_month = {
        'января': 1,
        'февраля': 2,
        'марта': 3,
        'апреля': 4,
        'мая': 5,
        'июня': 6,
        'июля': 7,
        'августа': 8,
        'сентября': 9,
        'октября': 10,
        'ноября': 11,
        'декабря':12
    }
    
    # создали словарь для объявления
    row = {
        'url_car': url_car,
        'car_make': None,
        'car_model': None,
        'car_gen': None,
        'car_type': None,
        'car_compl': None,
        'ann_date': None,
        'ann_id': None,
        'car_price': None,
        'ann_city': None,
        'link_cpl': None,
        'avail': None,
        'year': None,
        'mileage': None,
        'color': None,
        'eng_size': None,
        'eng_power': None,
        'eng_power_kw': None,
        'eng_type': None,
        'pow_resrv': None,
        'options': None,
        'transmission': None,
        'drive': None,
        'st_wheel': None,
        'condition': None,
        'count_owner': None,
        'original_pts': None,
        'customs': None
        }

    # загрузили страницу с данными по автомобилю
    page_auto = requests.get(url_car, headers=headers)
        
    # получили иерархическое представление содержимого
    soup_auto = BeautifulSoup(page_auto.content, 'html.parser')

    # извлекли данные из блока CardBreadcrumbs
    CardBreadcrums = soup_auto.find('div', {'class': 'CardBreadcrumbs'})
    row['car_make'] = CardBreadcrums.find_all('a')[2].text
    row['car_model'] = CardBreadcrums.find_all('a')[3].text
    row['car_gen'] = CardBreadcrums.find_all('a')[4].text
    row['car_type'] = CardBreadcrums.find_all('a')[5].text
    row['car_compl'] = CardBreadcrums.find_all('a')[6].text

    # извлекли данные об объявлении
    ann_date = soup_auto.find('div', {'class': 'CardHead__infoItem CardHead__creationDate'}).text.strip().split()
    row['ann_date'] = str(date(year, dict_month[ann_date[1]], int(ann_date[0])))
    row['ann_id'] = soup_auto.find('div', {'class': 'CardHead__infoItem CardHead__id'}).text.strip()[2:]
    row['car_price'] = int(soup_auto.find('span', {'class': 'OfferPriceCaption__price'}).text.replace('\xa0','')[:-1])
    row['ann_city'] = soup_auto.find('span', {'class': 'MetroListPlace__regionName MetroListPlace_nbsp'}).text
        

    card_info = soup_auto.find('div', {'class': 'CardInfo-ateuv'})

    # записали ссылку на комплектацию
    for a in soup_auto.find_all('a'):
        if 'Характеристики модели' in a.text:
            row['link_cpl'] = a.get('href')
        
    # извлекли сведения об автомобиле
    row['avail'] = card_info.find('li', {'class': 'CardInfoRow CardInfoRow_availability'}).find_all('div')[1].text.replace('\xa0',' ')
    row['year'] = int(card_info.find('li', {'class': 'CardInfoRow CardInfoRow_year'}).find_all('div')[1].text)
    row['mileage'] = int(card_info.find('li', {'class': 'CardInfoRow CardInfoRow_kmAge'}).find_all('div')[1].text.replace('\xa0','')[:-3])
    row['color'] = card_info.find('li', {'class': 'CardInfoRow CardInfoRow_color'}).find_all('div')[1].text
    if card_info.find('li', {'class': 'CardInfoRow CardInfoRow_engine'}).find_all('div')[1].text.split(' / ')[2] == 'Электро':
        # eng_size остаётся None для электромобилей
        eng_data = card_info.find('li', {'class': 'CardInfoRow CardInfoRow_engine'}).find_all('div')[1].text.split(' / ')
        row['eng_power_kw'] = int(eng_data[1].split()[0])
        row['eng_power'] = int(HP_per_kW * row['eng_power_kw'])        
        row['pow_resrv'] = card_info.find('li', {'class': 'CardInfoRow CardInfoRow_electricRange'}).find_all('div')[1].text.split()[0]
    else:
        # pow_resrv остаётся None для авто с ДВС
        eng_data = card_info.find('li', {'class': 'CardInfoRow CardInfoRow_engine'}).find_all('div')[1].text.split(' / ')
        row['eng_size'] = float(eng_data[0].split()[0])
        row['eng_power'] = int(eng_data[1].split()[0])
        row['eng_power_kw'] = int(row['eng_power'] / HP_per_kW)
    row['eng_type'] = eng_data[2]
    row['options'] = card_info.find('li', {'class': 'CardInfoRow CardInfoRow_complectationOrEquipmentCount'}).find_all('div')[1].text.strip().replace('\xa0',' ')
    row['transmission'] = card_info.find('li', {'class': 'CardInfoRow CardInfoRow_transmission'}).find_all('div')[1].text
    row['drive'] = card_info.find('li', {'class': 'CardInfoRow CardInfoRow_drive'}).find_all('div')[1].text
    row['st_wheel'] = card_info.find('li', {'class': 'CardInfoRow CardInfoRow_wheel'}).find_all('div')[1].text
    row['condition'] = card_info.find('li', {'class': 'CardInfoRow CardInfoRow_state'}).find_all('div')[1].text
    try:
        row['count_owner'] = card_info.find('li', {'class': 'CardInfoRow CardInfoRow_ownersCount'}).find_all('div')[1].text.replace('\xa0',' ')
    except:
        pass
    try:
        row['original_pts'] = card_info.find('li', {'class': 'CardInfoRow CardInfoRow_pts'}).find_all('div')[1].text
    except:
        pass
    try:
        row['customs'] = card_info.find('li', {'class': 'CardInfoRow CardInfoRow_customs'}).find_all('div')[1].text
    except:
        pass

    if row['link_cpl'] in list(map(lambda x: x['url_compl'], complectations)):

        # добавили сведения из ранее загруженной комплектации
        for record in complectations:
            if row['link_cpl'] == record['url_compl']:
                row = {**row, **record}
    else:       
        # добавили сведения из комплектации
        record = get_complectation(row['link_cpl'])
        complectations.append(record)
        with open('data_comp_dmitrii.txt', 'a') as data_comp:
            print(record, file=data_comp)
        row = {**row, **record}
    
    # передаем словарь
    return row

In [6]:
# Определим функцию, которая создаёт и возвращает список марок автомобилей,
# первая буква названия которых находится в диапазоне от a до b (параметры)
def get_page_makes(a, b):

    # изготовили ссылку на страницу списка с объявлениями
    url = 'https://auto.ru/catalog/cars/'

    # немного подождали
    sleep()

    # сходили по ней
    page_list = requests.get(url, headers=headers)

    # получили иерархическое представление содержимого
    soup = BeautifulSoup(page_list.content, 'html.parser')

    # ищем тут CatalogFilterSearchList__hidden-stSPU
    card_info = soup.find('div', {'class': 'CatalogFilterSearchList__hidden-stSPU'})

    # нашли в нём марки авто, записали в список
    name_makes = []
    for link in card_info.find_all('a'):
        if link.get('href'):
            name_makes.append(link.get('href').split('/')[-2])
    
    # убрали дубли
    name_makes = list(set(name_makes))
    
    # отсортировали по афавиту
    name_makes.sort()

    # оставили марки из диапазона
    name_makes = list(filter(lambda x: (ord(x[0]) >= ord(a) and ord(x[0]) <= ord(b)), name_makes))
    
    return name_makes

In [7]:
# Определим функцию, которая загружает объявления с одной
# из страниц выдачи объявлений определённой марки автомобилей
def get_page(make, p):
    # изготовили ссылку на страницу списка с объявлениями
    url_base = f'https://auto.ru/cars/{make}/used/'
    if p ==1:
        url = f'{url_base}'
    else:
        url = f'{url_base}?page={p}'

    # немного подождали
    sleep()

    # сходили по ней
    page_list = requests.get(url, headers=headers)

    # получили иерархическое представление содержимого
    soup = BeautifulSoup(page_list.content, 'html.parser')

    # нашли в нём ссылки на авто, записали в список
    urls = []
    for link in soup.find_all('a'):
        if link.get('href') != None and 'cars/used/sale' in link.get('href') and link.get('href') not in urls:
            urls.append(link.get('href'))

    # прошли по ссылкам записали данные в список словарей
    info = []
    mistakes = []

    # включаем счетчики
    success = 0
    failure = 0
    t = tqdm(urls, desc=f'Загрузка {name_make}/{page}. Успешно: {success}. Ошибок: {failure}')
    for link in t:
        try:
            info.append(get_car(link))
            success += 1
        except:
            mistakes.append(link)
            failure += 1
        t.set_description(f'Загрузка {name_make}/{page}. Успешно: {success}. Ошибок: {failure}')
        t.refresh()

    return info, success, failure

In [8]:
# Определим функцию, которая определяет количество объявлений
# для определённой марки автомобиля
def get_page_models(name_make):

    # изготовили ссылку на страницу списка с объявлениями
    url = f'https://auto.ru/cars/{name_make}/used/'
    
    # немного подождали
    sleep()
    
    # сходили по ней
    page_list = requests.get(url, headers=headers)

    # получили иерархическое представление содержимого
    soup = BeautifulSoup(page_list.content, 'html.parser')

    # Ищем тут ButtonWithLoader__inner
    model_info = soup.find('div', {'class': 'ButtonWithLoader__inner'})

    # определили количество объявлений
    summ = int(''.join(model_info.text.split()[1:-1]))
        
    return summ

In [9]:
# Определим функцию, которая открывает и считывает данные
# из текстового файла и возвращает датасет
def open_data(file_name):
    data_cars = []
    with open(f'{file_name}', 'r') as data:
        ann_ids = set()
        for record in data:
            res = {}
            for paars in record[1:-2].split(", '"):
                key, value = paars.split("':")
                value = value.replace("'","").strip()
                if value == 'None':
                    value = None
                res[key.replace("'","").strip()] = value
            if res['ann_id'] not in ann_ids:
                data_cars.append(res)
                ann_ids.add(res['ann_id'])
    return pd.DataFrame(data_cars)

## Сбор данных с сайта

In [23]:
# Загрузим имеющиеся сведения о комплектациях
data_comp = 'data_comp_dmitrii.txt'
try:
    with open('log_dmitrii.txt', 'a') as log:
        complectations = open_complectation(data_comp)
        print(f'{datetime.date.today().isoformat()} {datetime.datetime.now().time()} Загружен файл с комплектацией {data_comp}', file=log)
except (FileNotFoundError, PermissionError):
    complectations = []
    with open('log_dmitrii.txt', 'a') as log:
        print(f'{datetime.date.today().isoformat()} {datetime.datetime.now().time()} Не был загружен файл с комплектацией {data_comp}', file=log)

In [38]:
# Загружаем наименования марок автомобилей
s_letter = 'v'
f_letter = 'z'
name_makes = get_page_makes(s_letter, f_letter)

# включаем счетчики
success_all = 0
failure_all = 0

# Запускаем цикл с перебором марок автомобилей
t_make = tqdm(name_makes, desc=f'Загрузка {s_letter} - {f_letter}. Успешно: {success_all}. Ошибок: {failure_all}')
for name_make in t_make:

    # Вычисляем количество страниц для загрузки по марке (не более 100)
    try:
        count_page = get_page_models(name_make) // 38 + 2
        count_page = min(count_page, 100)
    except:
        count_page = 1

    # включаем счетчики
    success_make = 0
    failure_make = 0

    # Запускаем цикл с перебором страниц марки
    if name_make == 'volvo':
        start_page = 5
    elif name_make in ['vauxhall', 'vaz', 'venucia', 'vgv', 'volkswagen']:
        start_page = count_page
    else:
        start_page = 1
    t = tqdm(range(start_page, count_page), desc=f'Загрузка {name_make}. Успешно: {success_make}. Ошибок: {failure_make}')
    for page in t:
        try:
            # Загружаем сведения об автомобиле
            date_page = get_page(name_make, page)
            success_make += date_page[1]
            success_all += date_page[1]
            failure_make += date_page[2]
            failure_all += date_page[2]

            # Обновляем информацию в прогресс-баре
            t.set_description(f'Загрузка {name_make}. Успешно: {success_make}. Ошибок: {failure_make}')
            t.refresh()
            with open('data_dmitrii.txt', 'a') as data:
                for record in date_page[0]:
                    print(record, file=data)
            with open('log_dmitrii.txt', 'a') as log:
                print(f'{datetime.date.today().isoformat()} {datetime.datetime.now().time()} Успешно загружена страница {name_make}/{page}. Число добавленных записей {len(date_page[0])}', file=log)
        except:
            with open('log_dmitrii.txt', 'a') as log:
                print(f'{datetime.date.today().isoformat()} {datetime.datetime.now().time()} Не была загружена страница {name_make}/{page}. Число добавленных записей {0}', file=log)

    
    # Обновляем информацию в прогресс-баре   
    t_make.set_description(f'Загрузка {s_letter} - {f_letter}. Успешно: {success_all}. Ошибок: {failure_all}')
    t_make.refresh()

Загрузка v - z. Успешно: 0. Ошибок: 0:   0%|          | 0/41 [00:00<?, ?it/s]

Загрузка vauxhall. Успешно: 0. Ошибок: 0: 0it [00:00, ?it/s]

Загрузка vaz. Успешно: 0. Ошибок: 0: 0it [00:00, ?it/s]

Загрузка vector. Успешно: 0. Ошибок: 0: 0it [00:00, ?it/s]

Загрузка venturi. Успешно: 0. Ошибок: 0: 0it [00:00, ?it/s]

...

Загрузка zotye/2. Успешно: 0. Ошибок: 0:   0%|          | 0/38 [00:00<?, ?it/s]

Загрузка zotye/3. Успешно: 0. Ошибок: 0:   0%|          | 0/38 [00:00<?, ?it/s]

Загрузка zotye/4. Успешно: 0. Ошибок: 0:   0%|          | 0/5 [00:00<?, ?it/s]

Загрузка zx. Успешно: 0. Ошибок: 0:   0%|          | 0/1 [00:00<?, ?it/s]

Загрузка zx/1. Успешно: 0. Ошибок: 0:   0%|          | 0/14 [00:00<?, ?it/s]

## Разбор и сохранение загруженных данных

In [2]:
# Откроем имеющийся датасет, полученный ранее
# Зададим типы данных в колонках.
dtypes_of_data = {
            'url_car': str,
            'car_make': str,
            'car_model': str,
            'car_gen': str,
            'car_type': str,
            'car_compl': str,
            'ann_id': str,
            'car_price': float,
            'ann_city': str,
            'link_cpl': str,
            'avail': str,
            'year': int,
            'mileage': int,
            'color': str,
            'eng_size': float,
            'eng_power': float,
            'eng_power_kw': float,
            'eng_type': str,
            'pow_resrv': str,
            'options': str,
            'transmission': str,
            'drive': str,
            'st_wheel': str,
            'condition': str,
            'count_owner': str,
            'original_pts': str,
            'customs': str,
            'url_compl': str,
            'state_mark': str,
            'class_auto': str,
            'door_count': float, # могут быть пропуски
            'seat_count': str, # могут быть значения с диапазоном, пропуски
            'long': float,
            'width': float,
            'height': float,
            'clearence': str, # могут быть значения с диапазоном
            'v_bag': str, # могут быть значения с диапазоном, пропуски
            'v_tank': float,
            'curb_weight': float,
            'gross_weight': float,
            'front_brakes': str,
            'rear_brakes': str,
            'max_speed': float,
            'acceleration': float,
            'fuel_cons': str, # могут быть значения "-"
            'fuel_brand': str,
            'engine_loc1': str,
            'engine_loc2': str,
            'turbocharg': str,
            'max_torq': float,
             'cyl_count': float # Могут быть пропуски
        }
data = pd.read_csv(os.path.join(DATA_PATH, 'data_dmitrii.csv'),
                   dtype=dtypes_of_data, low_memory=False)
print(data.shape)
data.head()

(159508, 52)


,url_car,car_make,car_model,car_gen,car_type,car_compl,ann_date,ann_id,car_price,ann_city,...,rear_brakes,max_speed,acceleration,fuel_cons,fuel_brand,engine_loc1,engine_loc2,turbocharg,max_torq,cyl_count
0,https://auto.ru/cars/used/sale/abarth/500/1125...,Abarth,500,I Рестайлинг,Кабриолет,1.4 MT (180 л.с.),2024-10-30 00:00:00,1125082098,2700000.0,Обнинск,...,дисковые,225.0,6.7,4.9,АИ-95,переднее,поперечное,турбонаддув,250.0,4.0
1,https://auto.ru/cars/used/sale/abarth/500/1118...,Abarth,500,I,Кабриолет,1.4 MT (135 л.с.),2024-03-17 00:00:00,1118403975,1400000.0,Минск,...,дисковые,NaN,6.9,7.1,АИ-95,переднее,поперечное,турбонаддув,206.0,4.0
2,https://auto.ru/cars/used/sale/abarth/500/1125...,Abarth,500,I Рестайлинг,Кабриолет,1.4 AMT (180 л.с.),2024-09-19 00:00:00,1125077090,2990000.0,Санкт-Петербург,...,дисковые,225.0,6.9,4.7,АИ-95,переднее,поперечное,турбонаддув,250.0,4.0
3,https://auto.ru/cars/used/sale/abarth/500/1124...,Abarth,500,I,Хэтчбек 3 дв.,1.4 MT (135 л.с.),2024-08-15 00:00:00,1124579446,2100000.0,Москва,...,дисковые,205.0,7.9,5.4,АИ-98,переднее,поперечное,турбонаддув,206.0,4.0
4,https://auto.ru/cars/used/sale/abarth/500/1125...,Abarth,500,I Рестайлинг,Хэтчбек 3 дв.,1.4 AT (159 л.с.),2024-10-14 00:00:00,1125453984,1650000.0,Москва,...,дисковые,211.0,7.8,7.3,АИ-95,переднее,поперечное,турбонаддув,248.0,4.0


In [11]:
# Получим датасет из новой порции данных
new_data = open_data('data_dmitrii.txt')
new_data = new_data.astype(dtype=dtypes_of_data)
print(new_data.shape)
new_data.head()

(9427, 52)


,url_car,car_make,car_model,car_gen,car_type,car_compl,ann_date,ann_id,car_price,ann_city,...,rear_brakes,max_speed,acceleration,fuel_cons,fuel_brand,engine_loc1,engine_loc2,turbocharg,max_torq,cyl_count
0,https://auto.ru/cars/used/sale/uaz/hunter/1124...,УАЗ,Hunter,2003-2024,Внедорожник 5 дв.,315195 2.7 MT (128 л.с.) 4WD,2024-11-14,1124213448,899000.0,Таганрог,...,барабанные,130.0,NaN,NaN,АИ-92,переднее,продольное,нет,NaN,4.0
1,https://auto.ru/cars/used/sale/uaz/3151/112591...,УАЗ,3151,1985-2013,Внедорожник 5 дв.,31512 2.5 MT (78 л.с.) 4WD,2024-11-12,1125916067,300000.0,Нижний Новгород,...,барабанные,110.0,NaN,NaN,АИ-76,переднее,продольное,нет,NaN,4.0
2,https://auto.ru/cars/used/sale/uaz/patriot/112...,УАЗ,Patriot,I Рестайлинг 2,Внедорожник 5 дв.,2.7 MT (128 л.с.) 4WD,2024-11-14,1125872492,1100000.0,Санкт-Петербург,...,барабанные,150.0,NaN,NaN,АИ-92,переднее,продольное,нет,NaN,4.0
3,https://auto.ru/cars/used/sale/uaz/patriot/112...,УАЗ,Patriot,I Рестайлинг 2,Внедорожник 5 дв.,2.7 MT (135 л.с.) 4WD,2024-11-08,1125155071,814500.0,Ростов-на-Дону,...,барабанные,150.0,NaN,NaN,АИ-92,переднее,продольное,нет,NaN,4.0
4,https://auto.ru/cars/used/sale/uaz/patriot/112...,УАЗ,Patriot,I Рестайлинг 3,Внедорожник 5 дв.,2.7 MT (135 л.с.) 4WD,2024-11-10,1125885210,895000.0,Краснодар,...,барабанные,150.0,NaN,NaN,АИ-92,переднее,продольное,нет,NaN,4.0


In [3]:
# Датасет от коллеги
data_1= pd.read_csv(os.path.join(DATA_PATH, 'data_vladislav.csv'),
                   dtype=dtypes_of_data, low_memory=False)
print(data_1.shape)
data_1.head()

(25064, 52)


,url_car,car_make,car_model,car_gen,car_type,car_compl,ann_date,ann_id,car_price,ann_city,...,rear_brakes,max_speed,acceleration,fuel_cons,fuel_brand,engine_loc1,engine_loc2,turbocharg,max_torq,cyl_count
0,https://auto.ru/cars/used/sale/qoros/3/1125145...,Qoros,3,2014-2022,Седан,1.6 AMT (126 л.с.),2024-09-23,1125145948,1310000.0,Владивосток,...,дисковые,197.0,12.6,-,АИ-95,переднее,поперечное,нет,155.0,4.0
1,https://auto.ru/cars/used/sale/radar/rd6/11253...,Radar,RD6,2022-н.в.,Пикап Двойная кабина,63kWh Electro AT (200 кВт),2024-10-07,1125344393,2900000.0,Владивосток,...,дисковые,185.0,7.3,NaN,NaN,NaN,NaN,NaN,384.0,NaN
2,https://auto.ru/cars/used/sale/ram/1500/112464...,Ram,1500,V,Пикап Двойная кабина Crew Cab,Crew Cab 5.7 AT (395 л.с.) 4WD,2024-11-08,1124644054,11890000.0,Парголово,...,дисковые,206.0,6.7,12.4,АИ-95,переднее,продольное,нет,556.0,8.0
3,https://auto.ru/cars/used/sale/ram/1500/112479...,Ram,1500,IV (DS/DJ),Пикап Двойная кабина Crew Cab,Crew Cab 3.0d AT (240 л.с.) 4WD,2024-11-08,1124790535,3940000.0,Москва,...,дисковые,NaN,NaN,8.7,ДТ,переднее,продольное,турбонаддув,569.0,6.0
4,https://auto.ru/cars/used/sale/ram/1500/112587...,Ram,1500,V,Пикап Двойная кабина Quad Cab,Quad Cab 3.6 AT (305 л.с.) 4WD,2024-11-08,1125872042,3990000.0,Москва,...,дисковые,186.0,8.5,10.2,АИ-95,переднее,продольное,нет,364.0,6.0


In [4]:
# Датасет от другого коллеги
data_2= pd.read_csv(os.path.join(DATA_PATH, 'data_nikita.csv'),
                   dtype=dtypes_of_data, low_memory=False)
print(data_2.shape)
data_2.head()

(33473, 52)


,url_car,car_make,car_model,car_gen,car_type,car_compl,ann_date,ann_id,car_price,ann_city,...,rear_brakes,max_speed,acceleration,fuel_cons,fuel_brand,engine_loc1,engine_loc2,turbocharg,max_torq,cyl_count
0,https://auto.ru/cars/used/sale/lamborghini/uru...,Lamborghini,Urus,I Рестайлинг,Внедорожник 5 дв. SE,SE 4.0hyb AT (800 л.с.) 4WD,2024-11-08,1123486824,52000000.0,Москва,...,керамические,312.0,3.4,NaN,АИ-98,переднее,продольное,турбонаддув,950.0,8.0
1,https://auto.ru/cars/used/sale/lamborghini/hur...,Lamborghini,Huracán,I Рестайлинг,Купе Sterrato,Sterrato 5.2 AMT (610 л.с.) 4WD,2024-11-12,1123411674,56300000.0,Балашиха,...,керамические,260.0,3.4,-,АИ-98,центральное,NaN,нет,560.0,10.0
2,https://auto.ru/cars/used/sale/lamborghini/cou...,Lamborghini,Countach,1974-1991,Купе,S Quattrovalvole 5.2 MT (455 л.с.),2024-11-10,1125778471,99000000.0,Москва,...,дисковые,295.0,4.9,-,АИ-92,центральное,NaN,нет,500.0,12.0
3,https://auto.ru/cars/used/sale/lamborghini/hur...,Lamborghini,Huracán,I,Купе,5.2 AMT (610 л.с.) 4WD,2024-11-09,1125876275,17650000.0,Москва,...,керамические,325.0,3.2,9.4,АИ-98,центральное,NaN,нет,560.0,10.0
4,https://auto.ru/cars/used/sale/lamborghini/hur...,Lamborghini,Huracán,I Рестайлинг,Купе,5.2 AMT (640 л.с.) 4WD,2024-11-15,1125968621,34990000.0,Москва,...,керамические,325.0,2.9,10.3,АИ-98,центральное,NaN,нет,600.0,10.0


In [5]:
# Исправим название столбца width
data_1.columns=data.columns
data_2.columns=data.columns

In [6]:
# Объединим имеющийся датасет с новыми
data = pd.concat([data, data_1, data_2], axis=0).reset_index(drop=True)
print(data.shape)
data.head()

(218045, 52)


,url_car,car_make,car_model,car_gen,car_type,car_compl,ann_date,ann_id,car_price,ann_city,...,rear_brakes,max_speed,acceleration,fuel_cons,fuel_brand,engine_loc1,engine_loc2,turbocharg,max_torq,cyl_count
0,https://auto.ru/cars/used/sale/abarth/500/1125...,Abarth,500,I Рестайлинг,Кабриолет,1.4 MT (180 л.с.),2024-10-30 00:00:00,1125082098,2700000.0,Обнинск,...,дисковые,225.0,6.7,4.9,АИ-95,переднее,поперечное,турбонаддув,250.0,4.0
1,https://auto.ru/cars/used/sale/abarth/500/1118...,Abarth,500,I,Кабриолет,1.4 MT (135 л.с.),2024-03-17 00:00:00,1118403975,1400000.0,Минск,...,дисковые,NaN,6.9,7.1,АИ-95,переднее,поперечное,турбонаддув,206.0,4.0
2,https://auto.ru/cars/used/sale/abarth/500/1125...,Abarth,500,I Рестайлинг,Кабриолет,1.4 AMT (180 л.с.),2024-09-19 00:00:00,1125077090,2990000.0,Санкт-Петербург,...,дисковые,225.0,6.9,4.7,АИ-95,переднее,поперечное,турбонаддув,250.0,4.0
3,https://auto.ru/cars/used/sale/abarth/500/1124...,Abarth,500,I,Хэтчбек 3 дв.,1.4 MT (135 л.с.),2024-08-15 00:00:00,1124579446,2100000.0,Москва,...,дисковые,205.0,7.9,5.4,АИ-98,переднее,поперечное,турбонаддув,206.0,4.0
4,https://auto.ru/cars/used/sale/abarth/500/1125...,Abarth,500,I Рестайлинг,Хэтчбек 3 дв.,1.4 AT (159 л.с.),2024-10-14 00:00:00,1125453984,1650000.0,Москва,...,дисковые,211.0,7.8,7.3,АИ-95,переднее,поперечное,турбонаддув,248.0,4.0


In [7]:
# Посмотрим общие сведения
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 218045 entries, 0 to 218044
Data columns (total 52 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   url_car       218045 non-null  object 
 1   car_make      218045 non-null  object 
 2   car_model     218045 non-null  object 
 3   car_gen       218045 non-null  object 
 4   car_type      218045 non-null  object 
 5   car_compl     218045 non-null  object 
 6   ann_date      218045 non-null  object 
 7   ann_id        218045 non-null  object 
 8   car_price     218045 non-null  float64
 9   ann_city      218045 non-null  object 
 10  link_cpl      218045 non-null  object 
 11  avail         218045 non-null  object 
 12  year          218045 non-null  int64  
 13  mileage       218045 non-null  int64  
 14  color         218045 non-null  object 
 15  eng_size      215519 non-null  float64
 16  eng_power     218045 non-null  float64
 17  eng_power_kw  13756 non-null   float64
 18  eng_

In [8]:
# Приведём дату к типу datetime
data['ann_date'] = pd.to_datetime(data['ann_date'].str[:10], format='%Y-%m-%d')
data['ann_date'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 218045 entries, 0 to 218044
Series name: ann_date
Non-Null Count   Dtype         
--------------   -----         
218045 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 1.7 MB


In [9]:
# Убедимся в отсутствии полных дубликатов
print(f'Полных дубликатов {data.duplicated(keep='first').sum()}')

Полных дубликатов 1275


In [10]:
# Избавимся от полных дубликатов
data.drop_duplicates(inplace=True)

In [11]:
# Проверим
data.duplicated(keep="first").sum()

np.int64(0)

In [12]:
# Проверим полные дубликаты без 'url_car', 'ann_id', 'ann_date', 'link_cpl', 'url_compl'
# т.е автомобили с одинаковыми характеристиками и ценой
data.drop(['url_car', 'ann_id', 'ann_date', 'link_cpl', 'url_compl'], axis=1).duplicated(keep='first').sum()

np.int64(187)

In [13]:
# Рассмотрим такие дубликаты
data[data.drop(
    ['url_car', 'ann_id', 'ann_date', 'link_cpl', 'url_compl'], axis=1
).duplicated(keep=False)].sort_values(by=['car_make', 'ann_date'])

,url_car,car_make,car_model,car_gen,car_type,car_compl,ann_date,ann_id,car_price,ann_city,...,rear_brakes,max_speed,acceleration,fuel_cons,fuel_brand,engine_loc1,engine_loc2,turbocharg,max_torq,cyl_count
245,https://auto.ru/cars/used/sale/aito/m5/1125454...,Aito,M5,2022-н.в.,Внедорожник 5 дв.,1.5hyb AT (496 л.с.) 4WD,2024-11-03,1125454036,5250000.0,Ростов-на-Дону,...,дисковые,210.0,4.6,NaN,АИ-95,переднее,поперечное,турбонаддув,675.0,4.0
159463,https://auto.ru/cars/used/sale/aito/m5/1124954...,Aito,M5,2022-н.в.,Внедорожник 5 дв.,1.5hyb AT (496 л.с.) 4WD,2024-11-04,1124954435,5250000.0,Ростов-на-Дону,...,дисковые,210.0,4.6,NaN,АИ-95,переднее,поперечное,турбонаддув,675.0,4.0
159465,https://auto.ru/cars/used/sale/aito/m5/1124591...,Aito,M5,2022-н.в.,Внедорожник 5 дв.,1.5hyb AT (496 л.с.) 4WD,2024-11-04,1124591923,5250000.0,Ростов-на-Дону,...,дисковые,210.0,4.6,NaN,АИ-95,переднее,поперечное,турбонаддув,675.0,4.0
159466,https://auto.ru/cars/used/sale/aito/m5/1124115...,Aito,M5,2022-н.в.,Внедорожник 5 дв.,1.5hyb AT (496 л.с.) 4WD,2024-11-04,1124115203,5250000.0,Ростов-на-Дону,...,дисковые,210.0,4.6,NaN,АИ-95,переднее,поперечное,турбонаддув,675.0,4.0
6646,https://auto.ru/cars/used/sale/byd/yangwang_u8...,BYD,Yangwang U8,2023-н.в.,Внедорожник 5 дв.,2.0hyb AT (1197 л.с.) 4WD,2024-11-03,1125778335,19999999.0,Москва,...,дисковые,NaN,3.6,NaN,АИ-95,переднее,поперечное,турбонаддув,1280.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100209,https://auto.ru/cars/used/sale/zeekr/001/11257...,Zeekr,001,I Рестайлинг,Лифтбек,100 kWh Electro AT (580 кВт) 4WD,2024-11-22,1125743401,6588000.0,Москва,...,дисковые,240.0,3.3,NaN,NaN,NaN,NaN,NaN,810.0,NaN
29189,https://auto.ru/cars/used/sale/gaz/12/11210370...,ГАЗ,12 ЗИМ,1948-1960,Седан,3.5 MT (90 л.с.),2024-10-25,1121037096,3800000.0,Уссурийск,...,барабанные,120.0,37.0,NaN,АИ-80,переднее,продольное,нет,215.0,6.0
29445,https://auto.ru/cars/used/sale/gaz/12/11257138...,ГАЗ,12 ЗИМ,1948-1960,Седан,3.5 MT (90 л.с.),2024-10-31,1125713838,3800000.0,Уссурийск,...,барабанные,120.0,37.0,NaN,АИ-80,переднее,продольное,нет,215.0,6.0
203584,https://auto.ru/cars/used/sale/moscvich/3/1125...,Москвич,3,2022-2024,Внедорожник 5 дв.,1.5 CVT (136 л.с.),2024-10-16,1125489141,1800000.0,Ростов-на-Дону,...,дисковые,170.0,NaN,5.5,АИ-92,переднее,поперечное,турбонаддув,200.0,4.0


In [14]:
# Удалим дубликаты, оставив последнее по времени объявление
data = data.sort_values(by='ann_date')
data = data[~data.drop(
    ['url_car', 'ann_id', 'ann_date', 'link_cpl', 'url_compl'],
    axis=1
).duplicated(keep='last')]
data.sort_index(inplace=True)

In [15]:
# Проверим
data.drop(['url_car', 'ann_id', 'ann_date', 'link_cpl', 'url_compl'], axis=1).duplicated(keep='first').sum()

np.int64(0)

In [16]:
# Проверим дубликаты по ann_id
print(f'Дубликаты по ann_id {data['ann_id'].duplicated(keep="first").sum()}')

Дубликаты по ann_id 52661


In [17]:
# Создадим столбец с количеством пропусков по объекту
data['isnasum'] = data.isna().sum(axis=1)

In [18]:
# Посмотрим дубликаты по ann_id
data[data['ann_id'].duplicated(keep=False)].sort_values(by=['ann_id', 'isnasum'])

,url_car,car_make,car_model,car_gen,car_type,car_compl,ann_date,ann_id,car_price,ann_city,...,max_speed,acceleration,fuel_cons,fuel_brand,engine_loc1,engine_loc2,turbocharg,max_torq,cyl_count,isnasum
49620,https://auto.ru/cars/used/sale/mazda/6_mps/100...,Mazda,6 MPS,2005-2008,Седан,2.3 MT (260 л.с.) 4WD,2024-01-30,1000114784,990000.0,Москва,...,240.0,6.6,8.0,АИ-95,переднее,поперечное,турбонаддув,380.0,4.0,2
196272,https://auto.ru/cars/used/sale/mazda/6_mps/100...,Mazda,6 MPS,2005-2008,Седан,2.3 MT (260 л.с.) 4WD,2024-01-30,1000114784,990000.0,Москва,...,240.0,6.6,8.0,АИ-95,переднее,поперечное,турбонаддув,380.0,4.0,2
55843,https://auto.ru/cars/used/sale/mitsubishi/paje...,Mitsubishi,Pajero Sport,I Рестайлинг,Внедорожник 5 дв.,2.5d MT (133 л.с.) 4WD,2024-02-17,1008953341,900000.0,Минск,...,151.0,18.0,8.3,ДТ,переднее,продольное,турбонаддув,280.0,4.0,2
202071,https://auto.ru/cars/used/sale/mitsubishi/paje...,Mitsubishi,Pajero Sport,I Рестайлинг,Внедорожник 5 дв.,2.5d MT (133 л.с.) 4WD,2024-02-17,1008953341,900000.0,Минск,...,151.0,18.0,8.3,ДТ,переднее,продольное,турбонаддув,280.0,4.0,2
56911,https://auto.ru/cars/used/sale/moscvich/svyato...,Москвич,Святогор,1997-2002,Хэтчбек 5 дв.,214100 1.6 MT (76 л.с.),2024-11-25,1025488297,123000.0,Санкт-Петербург,...,156.0,15.5,NaN,АИ-95,переднее,продольное,нет,121.0,4.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217923,https://auto.ru/cars/used/sale/porsche/panamer...,Porsche,Panamera,III,Лифтбек Executive,4 E-Hybrid Executive 2.9hyb AMT (470 л.с.) 4WD,2024-11-21,1126049324,20979000.0,Владивосток,...,280.0,4.2,NaN,АИ-98,переднее,продольное,турбонаддув,650.0,6.0,3
213456,https://auto.ru/cars/used/sale/porsche/macan/1...,Porsche,Macan,I,Внедорожник 5 дв.,S Diesel 3.0d AMT (245 л.с.) 4WD,2024-11-21,1126053771,3950000.0,Пермь,...,230.0,6.3,5.9,ДТ,переднее,продольное,турбонаддув,580.0,6.0,2
217497,https://auto.ru/cars/used/sale/porsche/macan/1...,Porsche,Macan,I,Внедорожник 5 дв.,S Diesel 3.0d AMT (245 л.с.) 4WD,2024-11-21,1126053771,3950000.0,Пермь,...,230.0,6.3,5.9,ДТ,переднее,продольное,турбонаддув,580.0,6.0,2
56923,https://auto.ru/cars/used/sale/moscvich/svyato...,Москвич,Святогор,1997-2002,Хэтчбек 5 дв.,214122 1.7 MT (85 л.с.),2024-08-28,46182926,380000.0,Москва,...,160.0,14.0,7.1,АИ-92,переднее,продольное,нет,133.0,4.0,2


In [19]:
# Уберём дубликаты по ann_id, оставив объекты с наименьшим кол-вом пропусков
data = data.sort_values(by=['ann_id', 'isnasum'])
data.drop(['isnasum'], axis=1, inplace=True)
data = data[~data['ann_id'].duplicated(keep='first')].copy()
data.sort_index(inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 163922 entries, 0 to 218043
Data columns (total 52 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   url_car       163922 non-null  object        
 1   car_make      163922 non-null  object        
 2   car_model     163922 non-null  object        
 3   car_gen       163922 non-null  object        
 4   car_type      163922 non-null  object        
 5   car_compl     163922 non-null  object        
 6   ann_date      163922 non-null  datetime64[ns]
 7   ann_id        163922 non-null  object        
 8   car_price     163922 non-null  float64       
 9   ann_city      163922 non-null  object        
 10  link_cpl      163922 non-null  object        
 11  avail         163922 non-null  object        
 12  year          163922 non-null  int64         
 13  mileage       163922 non-null  int64         
 14  color         163922 non-null  object        
 15  eng_size      162104 n

In [20]:
# Проверим дубликаты по ann_id
print(f'Дубликаты по ann_id {data['ann_id'].duplicated(keep="first").sum()}')

Дубликаты по ann_id 0


In [21]:
# Сбросим индексацию
data.reset_index(drop=True, inplace=True)

In [24]:
# Заменим в колонке fuel_cons "-" на np.nan
data.loc[data['fuel_cons'] == '-', 'fuel_cons'] = np.nan
data['fuel_cons'] = data['fuel_cons'].astype(float)

In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163922 entries, 0 to 163921
Data columns (total 52 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   url_car       163922 non-null  object        
 1   car_make      163922 non-null  object        
 2   car_model     163922 non-null  object        
 3   car_gen       163922 non-null  object        
 4   car_type      163922 non-null  object        
 5   car_compl     163922 non-null  object        
 6   ann_date      163922 non-null  datetime64[ns]
 7   ann_id        163922 non-null  object        
 8   car_price     163922 non-null  float64       
 9   ann_city      163922 non-null  object        
 10  link_cpl      163922 non-null  object        
 11  avail         163922 non-null  object        
 12  year          163922 non-null  int64         
 13  mileage       163922 non-null  int64         
 14  color         163922 non-null  object        
 15  eng_size      162

In [27]:
# Сохраним расширенный датасет, возможно, поменяв имя файла
data.to_csv(os.path.join(DATA_PATH, input('file_name?')), index=False)

In [24]:
# Преобразуем данные о комплектациях в датасет
complectations_df = pd.DataFrame(complectations)
complectations_df

,url_compl,state_mark,class_auto,door_count,seat_count,long,width,height,clearence,v_bag,...,rear_brakes,max_speed,acceleration,fuel_cons,fuel_brand,engine_loc1,engine_loc2,turbocharg,max_torq,cyl_count
0,https://auto.ru/catalog/cars/vaz/2105/2307266/...,Россия,C,4,5,4130,1620,1446,154,None,...,барабанные,148,None,None,АИ-95,переднее,продольное,нет,None,4
1,https://auto.ru/catalog/cars/vaz/2101/6256978/...,Россия,B,4,5,4043,1611,1440,170,None,...,барабанные,140,None,None,АИ-92,переднее,продольное,нет,None,4
2,https://auto.ru/catalog/cars/vaz/2105/2307266/...,Россия,C,4,5,4130,1620,1446,170,None,...,барабанные,150,None,None,АИ-92,переднее,продольное,нет,None,4
3,https://auto.ru/catalog/cars/vaz/2105/2307266/...,Россия,C,4,5,4130,1620,1446,170,None,...,барабанные,145,None,None,АИ-92,переднее,продольное,нет,None,4
4,https://auto.ru/catalog/cars/vaz/2105/2307266/...,Россия,C,4,5,4130,1620,1446,170,None,...,барабанные,150,None,None,АИ-95,переднее,продольное,нет,None,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5246,https://auto.ru/catalog/cars/zotye/sr9/2090977...,Китай,D,5,5,4744,1929,1647,161,None,...,дисковые,None,None,None,АИ-95,переднее,поперечное,турбонаддув,None,4
5247,https://auto.ru/catalog/cars/zx/grand_tiger/20...,Китай,J,4,5,5310,1812,1735,202,None,...,барабанные,140,None,None,АИ-95,переднее,продольное,нет,None,4
5248,https://auto.ru/catalog/cars/zx/landmark/20351...,Китай,J,5,5,4768,1780,1830,202,None,...,барабанные,140,None,None,АИ-95,переднее,продольное,нет,None,4
5249,https://auto.ru/catalog/cars/zx/landmark/20351...,Китай,J,5,5,4768,1780,1830,202,None,...,барабанные,140,None,None,АИ-95,переднее,продольное,нет,None,4


In [25]:
# Обновим файл с комплектациями
complectations_df.to_csv(os.path.join(DATA_PATH, 'complectations_dmitrii.csv'), index=False)